In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

##Word-Document and Word-Word Matrices

In [ ]:
# Example documents (Shakespeare plays)
documents = {
    "As You Like It": "battle good fool wit love forest magic",
    "Twelfth Night": "good fool wit love comedy mistaken identity",
    "Julius Caesar": "battle battle battle good fool war rome politics",
    "Henry V": "battle battle battle battle good wit war king england"
}

['battle',
 'comedy',
 'england',
 'fool',
 'forest',
 'good',
 'identity',
 'king',
 'love',
 'magic',
 'mistaken',
 'politics',
 'rome',
 'war',
 'wit']

In [ ]:
def create_term_document_matrix(documents):
  """ Creates term-document matrix
  Rows represent words (terms)
  Columns represent docs
  Cells have the frequencies (counts)"""

  all_the_words = set()

  for doc in documents.values():
    all_the_words.update(doc.split())

  vocab = sorted(all_the_words)
  print(vocab)

  matrix = []

  for word in vocab:
    row = []

    for doc_name, doc_text in documents.items():
      count = doc_text.split().count(word)
      row.append(count)

    matrix.append(row)

  print(matrix)


  df = pd.DataFrame(matrix,
                    index=vocab,
                    columns=list(documents.keys()))



[[1, 0, 3, 4],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [1, 1, 1, 0],
 [1, 0, 0, 0],
 [1, 1, 1, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [1, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 1],
 [1, 1, 0, 1]]